In [1]:
!wget https://www.manythings.org/anki/rus-eng.zip
!unzip rus-eng.zip

--2023-03-31 14:03:18--  https://www.manythings.org/anki/rus-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15374406 (15M) [application/zip]
Saving to: ‘rus-eng.zip’

rus-eng.zip         100%[===================>]  14.66M  6.53MB/s    in 2.2s    

2023-03-31 14:03:22 (6.53 MB/s) - ‘rus-eng.zip’ saved [15374406/15374406]

Archive:  rus-eng.zip
  inflating: rus.txt                 
  inflating: _about.txt              


In [1]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks import LearningRateMonitor
import sys, os

sys.path.append(os.path.join(os.getcwd(), "./src"))

from models.rnn import Seq2SeqRNN
from data.datamodule import DataManager

In [2]:
eng_prefixes = (
    "i am ",
    "i m ",
    "he is",
    "he s ",
    "she is",
    "she s ",
    "you are",
    "you re ",
    "we are",
    "we re ",
    "they are",
    "they re ",
)

def filter_func(x):
    len_filter = lambda x: max(len(x[0].split(" ")), len(x[1].split(" "))) <= 5
    prefix_filter = lambda x: x[0].startswith(eng_prefixes)
    return len_filter(x) and prefix_filter(x)

config = {
    "batch_size": 256,          # <--- size of batch
    "num_workers": 16,          # <--- num cpu to use in dataloader
    "filter": filter_func,      # <--- callable obj to filter data  
    "filename": "./rus.txt",    # <--- path to file with sentneces
    "lang1": "en",              # <--- name of the first lang    
    "lang2": "ru",              # <--- name of the second lang
    "reverse": False,           # <--- direct or reverse order in pairs
    "train_size": 0.8,          # <--- ratio of data pairs to use in train
    "run_name": "tutorial",     # <--- run name to logger and checkpoints
    "quantile": 0.95,           # <--- (1 - quantile) longest sentences will be removed
}

In [3]:
# Data manager
dm = DataManager(config)
dm.prepare_data()
dm.setup()

input_lang_n_words = dm.input_lang_n_words
output_lang_n_words = dm.output_lang_n_words

Reading from file: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 464010/464010 [00:06<00:00, 72282.88it/s]


In [4]:
model = Seq2SeqRNN(
    encoder_vocab_size=input_lang_n_words,
    encoder_embedding_size=256,
    decoder_embedding_size=256,
    decoder_output_size=output_lang_n_words,
    lr=1e-3,
    output_lang_index2word=dm.train_dataset.output_lang.index2word,
)

In [9]:
help(TensorBoardLogger)

Help on class TensorBoardLogger in module pytorch_lightning.loggers.tensorboard:

class TensorBoardLogger(pytorch_lightning.loggers.logger.Logger, lightning_fabric.loggers.tensorboard.TensorBoardLogger)
 |  TensorBoardLogger(save_dir: Union[str, pathlib.Path], name: Union[str, NoneType] = 'lightning_logs', version: Union[int, str, NoneType] = None, log_graph: bool = False, default_hp_metric: bool = True, prefix: str = '', sub_dir: Union[str, pathlib.Path, NoneType] = None, **kwargs: Any)
 |  
 |  Log to local file system in `TensorBoard <https://www.tensorflow.org/tensorboard>`_ format.
 |  
 |  Implemented using :class:`~tensorboardX.SummaryWriter`. Logs are saved to
 |  ``os.path.join(save_dir, name, version)``. This is the default logger in Lightning, it comes
 |  preinstalled.
 |  
 |  Example:
 |  
 |  .. testcode::
 |  
 |      from pytorch_lightning import Trainer
 |      from pytorch_lightning.loggers import TensorBoardLogger
 |  
 |      logger = TensorBoardLogger("tb_logs", n

In [6]:
# TB Logger
logger = TensorBoardLogger("lightning_logs", name=config["run_name"])

# Callbacks
checkpoint_callback = ModelCheckpoint(
    save_top_k=3,
    monitor="val_loss",
    mode="min",
    dirpath="runs/{}/".format(config["run_name"]),
    filename="{epoch:02d}-{step:d}-{val_loss:.4f}",
    verbose=True,
    every_n_epochs=1,
)
lr_monitor = LearningRateMonitor(logging_interval="step")

# Initialize a Trainer
trainer = pl.Trainer(
    accelerator="cuda",
    devices=[0],
    precision=16,
    max_epochs=50,
    min_epochs=1,
    callbacks=[lr_monitor, checkpoint_callback],
    check_val_every_n_epoch=1,
    logger=logger,
    log_every_n_steps=1,
)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [10]:
trainer.fit(model, dm)

Reading from file: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 464010/464010 [00:06<00:00, 73089.84it/s]
Missing logger folder: lightning_logs/tutorial
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type       | Params
-----------------------------------------
0 | encoder   | EncoderRNN | 1.1 M 
1 | decoder   | DecoderRNN | 3.3 M 
2 | criterion | NLLLoss    | 0     
-----------------------------------------
4.4 M     Trainable params
0         Non-trainable params
4.4 M     Total params
17.465    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/somov-od/nmt_venv/lib/python3.8/site-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


вы опять врёте ---> комнатах ожидании незамужние слепой
ты слишком расслаблен ---> полки знаменитый лишний противен
я всегда готова ---> выигрываю закончилось британец горький
мы остаёмся здесь ---> морепродукты избалованный безнравственна компаньон
у меня идет ---> знают убиваешь ортодонт праздную
ты идиот ---> полки наркоман жажда честолюбивы
он не женат ---> умна доступна романтик вечером
он невиновен ---> морепродукты очевидно прихожу комнате
я атеист ---> одарён надоел заказчик пляж
я иду за ---> музыканты музыканты утомил английском
мы с ним ---> бездарен устаю романтик подумываю
я тот кто ---> расстроена растерянности рулём зовут
я вполне счастлива ---> морепродукты очевидно прихожу комнате
ты очень грязная ---> полки наркоман жажда честолюбивы
я в своей ---> суеверен смс без пришёл


Training: 0it [00:00, ?it/s]

...

вы опять врёте ---> ты лжёте
ты слишком расслаблен ---> ты слишком слишком
я всегда готова ---> я всегда время
мы остаёмся здесь ---> мы чудаки
у меня идет ---> у меня сильное
ты идиот ---> ты взрослый
он не женат ---> он не
он невиновен ---> он оскорблён
я атеист ---> я агент
я иду за ---> я следую за
мы с ним ---> ему примерно сорока
я тот кто ---> я та
я вполне счастлива ---> я рад счастлив
ты очень грязная ---> ты очень одарена
я в своей ---> я в в


Epoch 48, global step 2009: 'val_loss' was not in top 3


Validation: 0it [00:00, ?it/s]

вы опять врёте ---> ты лжёте
ты слишком расслаблен ---> ты слишком доверчивы
я всегда готова ---> я всегда время
мы остаёмся здесь ---> мы кричим
у меня идет ---> у меня сильное
ты идиот ---> ты взрослый
он не женат ---> он не дома
он невиновен ---> он бездомный и
я атеист ---> я взрослый
я иду за ---> я следую за
мы с ним ---> он примерно твоего
я тот кто ---> я та кто
я вполне счастлива ---> я рад за
ты очень грязная ---> ты действительно одарена
я в своей ---> я в в


Epoch 49, global step 2050: 'val_loss' was not in top 3
`Trainer.fit` stopped: `max_epochs=50` reached.


Now you can see tensorboard logs in two ways: launch tensorboard extension in jupyter notebook or use CLI method.

CLI:
1. Tap `tensorboard --logdir=./lightning_logs --port=6006`
2. Forward selected port in the ssh connection if you are working remote else just open `localhost:6006` in the browser


Jupyter:
1. Load extension: `%load_ext tensorboard`
2. Launch built-in tensorboard: `%tensorboard --logdir=./lightning_logs`

In [12]:
%load_ext tensorboardX
%tensorboard --logdir=./lightning_logs

The tensorboardX module is not an IPython extension.


UsageError: Line magic function `%tensorboard` not found.


# Hints

#### Load model from checkpoint
Using `self.save_hyperparameters` in `__init__` body of `pl.LightningModule` allows to load model in this way:
```python
import pytorch_lightning as pl
model = Seq2SeqRNN.load_from_checkpoint('checkpoint.ckpt')
```

Or you can load checkpoint in natural pytorch way:
```python
model = Seq2SeqRNN(*args, **kwargs)
model.load_state_dict(torch.load('checkpoint.ckpt')['state_dict'])
```

#### Add custom metrics to logger
https://lightning.ai/docs/pytorch/stable/extensions/logging.html


#### Enable grad accumulation
In this example accumulation will be the following: 
1. from 0 to 15th epoch accumulate 4 batches
2. from 15th to 25th epoch accumulate 2 batches
3. from 25th epoch accumulate 1 batch

```python
from pytorch_lightning.callbacks import GradientAccumulationScheduler

accumulator = GradientAccumulationScheduler(scheduling={0: 4, 15: 2, 25: 1})
trainer = pl.Trainer(
    ...
    callbacks=[..., accumulator],
    ...
)
```

#### Configure learning rate scheduler

```python
def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
    lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode='min',
        factor=0.1,
        patience=0,
        threshold=1e-2,
        threshold_mode='rel',
        cooldown=0,
        min_lr=0,
        eps=1e-09,
        verbose=True
    )
    lr_dict = {
        "scheduler": lr_scheduler,
        "interval": "epoch",
        "frequency": 1,
        "monitor": "val_loss"
    }
    return [optimizer], [lr_dict]
```

#### Other logger: WandB
You can use famous weights&biases logger which is natively supports in pytorch-lightning:
https://lightning.ai/docs/pytorch/stable/api/lightning.pytorch.loggers.wandb.html#module-lightning.pytorch.loggers.wandb